In [53]:
import sqlite3
from PIL import Image
con = sqlite3.connect('food_image.db')
cur = con.cursor()


In [55]:
cur.executescript('''
    DROP TABLE IF EXISTS food_image;
    CREATE TABLE food_image(
        FOOD_NAME    TEXT,
        FOOD_IMAGE    TEXT
    );
''')


In [ ]:
# cur.executescript('''
#     DROP TABLE IF EXISTS IMAGE;
#     CREATE TABLE IMAGE(
#         FOOD_NAME    TEXT,
#         IMAGE    BLOB
#     );
# ''')

In [2]:
import requests
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep

headers = {'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}

def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    method = method.upper()
    if canfetch(url) == False:
        print('[Error] ' + url)
    else: #bot 접근 못하게 막은데있어어 else뺌
        try:
            resp = requests.request(method, url,
                   params=params if method=='GET' else {},
                   data=params if method=='POST' else {},
                   headers=headers)
            resp.raise_for_status()
        except HTTPError as e:
            if limit > 0 and e.response.status_code >= 500:
                print(limit)
                time.sleep(300) # => random, 1초에 1번
                resp = download(url, params, headers, method, limit-1)
            else:
                print('[{}] '.format(e.response.status_code) + url)
                print(e.response.status_code)
                print(e.response.reason)
                print(e.response.headers)
        return resp

In [16]:
resp = download('https://www.gettyimagesbank.com/s/?lv=&st=union&mi=2&q=%25EA%25B9%2580%25EC%25B9%2598%25EC%25B0%258C%25EA%25B0%259C&ssi=go',headers = headers, method='GET')
dom = BeautifulSoup(resp.text,'lxml')

<Response [200]>

In [65]:
flist = ['멍게비빔밥', '새우초밥', '연어초밥', '약초비빔밥', '해물덮밥', '월남쌈', '김치찌개', '꽁치김치찌개']

In [66]:
from selenium import webdriver
driver = webdriver.Chrome('/Users/Gyu/Desktop/mywork/KData/chromedriver')

for search_list in flist:
    driver.get('https://www.gettyimagesbank.com/s/?lv=&st=union&mi=2&q='+search_list+'&ssi=go')
    time.sleep(3)
    if not driver.find_elements_by_css_selector('img[class=imgThumb]'):
        continue
    for _ in driver.find_elements_by_css_selector('img[class=imgThumb]'):
                cur.execute('''
            INSERT INTO food_image(FOOD_NAME, FOOD_IMAGE)
            VALUES(?,?)''' ,[search_list,_.get_attribute('src')]
        )
con.commit()
driver.close()

In [ ]:
# from selenium import webdriver
# from urllib.request import urlopen
# driver = webdriver.Chrome('/Users/Gyu/Desktop/mywork/KData/chromedriver')

# for search_list in flist:
#     driver.get('https://www.gettyimagesbank.com/s/?lv=&st=union&mi=2&q='+search_list+'&ssi=go')
#     time.sleep(3)
#     if not driver.find_elements_by_css_selector('img[class=imgThumb]'):
#         continue
#     for _ in driver.find_elements_by_css_selector('img[class=imgThumb]'):
#         img = urlopen(_.get_attribute('src'))
#         break
#                 cur.execute('''
#             INSERT INTO IMAGE(FOOD_NAME, IMAGE)
#             VALUES(?,?)''' ,[search_list,urlopen(_.get_attribute('src'))]
#         )
# con.commit()
# driver.close()

In [63]:
driver.close()

In [4]:
cur.close()
con.close()

In [1]:
from selenium import webdriver
driver = webdriver.Chrome('/Users/Gyu/Desktop/mywork/KData/chromedriver')
driver.get('https://pixabay.com/ko/photos/search/')

In [52]:
driver.close()

In [51]:
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import time
flist = ['토마토','고추', '당근', '마늘', '무', '호박']
url = 'https://pixabay.com/ko/photos/search/'


for i in flist:
    driver.get(url + i + '/?cat=food')
    num_of_pagedowns = 4
    time.sleep(3)
    while num_of_pagedowns:
        
        try:
            body = driver.find_element_by_tag_name('body')
            body.send_keys(Keys.END)
            time.sleep(4)
            num_of_pagedowns -= 1
        
            for j,_ in enumerate(driver.find_elements_by_css_selector('img')):
                if _.get_attribute('srcset') !='':
                    cur.execute('''
                        INSERT INTO food_image(FOOD_NAME, FOOD_IMAGE)
                        VALUES(?,?)''' ,[i,_.get_attribute('srcset')]
                     )
            time.sleep(2)
            driver.find_element_by_xpath('//*[@id="content"]/div/a').click()
            time.sleep(3)
        except StaleElementReferenceException:
            pass
con.commit()


In [41]:
from selenium.webdriver.common.keys import Keys
from urllib.request import urlopen
from selenium.common.exceptions import StaleElementReferenceException
import time
flist = ['토마토', '사과', '딸기']
url = 'https://www.google.com/search?q='
detail = '&rlz=1C1NDCM_koKR830KR830&sxsrf=ALeKk02Fz_qw4R-HKvLq2RYYTPm1AoHutA:1596589422260&source=lnms&tbm=isch&sa=X&ved=2ahUKEwiRufaK74LrAhWDPXAKHfy-Ar8Q_AUoAXoECBoQAw&biw=1920&bih=842'


for i in flist:
    driver.get(url + i + detail)
    num_of_pagedowns = 1
    time.sleep(3)
    while num_of_pagedowns:
        
        
        body = driver.find_element_by_tag_name('body')
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        num_of_pagedowns -= 1
        
        for j,_ in enumerate(driver.find_elements_by_css_selector('img[class$=Q4LuWd]')[1:21]):
            if _.get_attribute('src') !='':
#               print(_.get_attribute('src'))
                try:
                    with urlopen(_.get_attribute('src')) as f:
                        with open('img/' + i + '_' + str(j)+'.jpg','wb') as h: 
                            img = f.read()
                            h.write(img)
                except:
                    with urlopen(_.get_attribute('data-src')) as f:
                        with open('img/' + i + '_' + str(j)+'.jpg','wb') as h: 
                            img = f.read()
                            h.write(img)
            time.sleep(2)
